In [11]:
!pip install pytorch-transformers
!pip install emoji
!pip install pyenchant
!pip install nltk
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import XLNetModel, XLNetTokenizer, XLNetForSequenceClassification
from pytorch_transformers import AdamW
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from torch import Tensor
from torch.nn import BCEWithLogitsLoss
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.metrics import f1_score, recall_score, precision_score, multilabel_confusion_matrix
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

  Using cached https://files.pythonhosted.org/packages/9e/54/04d88a59efa33fefb88133ceb638cdf754319030c28aadc5a379d82140ed/pyenchant-2.0.0.tar.gz
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


'Tesla P100-PCIE-16GB'

In [3]:
# Upload train.tsv and validation.tsv
from google.colab import files
uploaded = files.upload()

Saving validation.tsv to validation.tsv


In [0]:
# Represent labels as one-hot encoded vectors
train_df = pd.read_csv("train.tsv", delimiter='\t')
test_df = pd.read_csv("validation.tsv", delimiter='\t')
train_tweets = train_df.tweet.values
test_tweets = test_df.tweet.values
label_names=["anger","disgust","fear","joy","sadness","surprise"]
label_ids=[1,2,3,4,5,6]
train_labels=[]
for i in range(len(train_df)):
    train_label=[]
    for key in label_names:
        train_label.append(int(train_df.iloc[i][key]))
    train_labels.append(train_label)
test_labels=[]
for i in range(len(test_df)):
    test_label=[]
    for key in label_names:
        test_label.append(int(test_df.iloc[i][key]))
    test_labels.append(test_label)

In [5]:
# Pre-processing
import regex as re
import emoji
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')
ps = PorterStemmer()
def preprocess(tweets, negation_handling = True, links_handling = True, stopword=True):
  max = 0
  clean_tweets = []
  # NLTK stop words
  stop_words = set(stopwords.words('english')) 
  for tweet in tweets:
    if links_handling:
      # Delete links and mentions
      tweet = re.sub(r"(?:\@|https?\://)\S+", "", tweet)
    neg_feature = re.compile(r"^not$|never|[a-z]n't$")   # regular expression for not, n't and never
    negation = False
    # Tweet to list of words
    words=word_tokenize(tweet)
    if stopword:
    # Delete the stopwords
      words = [w for w in words if not w in stop_words]
    for i in range(len(words)): 
      if negation_handling:
        #Add NOT_ to every word after not, n't and never until punctuation
        if (words[i] not in (',', '.', '?', '!', ';')) & negation:
            words[i] = "NOT_" + words[i]
        if neg_feature.search(words[i]):
            negation = True
        if words[i] in (',', '.', '?', '!', ';'):
            negation = False
    tweet = ' '.join(word for word in words)
    clean_tweets.append(tweet)
  #Add begining and end of sentence tokens
  clean_tweets = [tweet + " [SEP] [CLS]" for tweet in clean_tweets]
  return clean_tweets

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
class XLNetForMultiLabelSequenceClassification(XLNetForSequenceClassification):
    r"""
        Method overriding of XLNetForSequenceClassification to adapt it to multi-label classification 
        Changes: labels vector is extended to the number labels instead of 1
    """

    def forward(self, input_ids, token_type_ids=None, input_mask=None, attention_mask=None,
                mems=None, perm_mask=None, target_mapping=None,
                labels=None, head_mask=None):
        transformer_outputs = self.transformer(input_ids, token_type_ids=token_type_ids,
                                               input_mask=input_mask, attention_mask=attention_mask,
                                               mems=mems, perm_mask=perm_mask, target_mapping=target_mapping,
                                               head_mask=head_mask)
        output = transformer_outputs[0]

        output = self.sequence_summary(output)
        logits = self.logits_proj(output)

        # Keep mems, hidden states, attentions if there are in it
        outputs = (logits,) + transformer_outputs[1:]

        if labels is not None:
            loss_fct = BCEWithLogitsLoss()
        #Changes: labels vector is extended to the number labels instead of 1
            loss = loss_fct(logits.view(-1, self.num_labels),
                            labels.view(-1, self.num_labels).type_as(logits.view(-1, self.num_labels)))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [0]:
def xlnet_model(train_tweets, test_tweets, train_labels, test_labels, epochs = 4, batch_size = 32, lr = 3e-5, T=0.5):
    """
    lr = learning rate
    T = probabilistic threshold
    """
    tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', do_lower_case=True)
    MAX_LEN = 128

    tokenized_train = [tokenizer.tokenize(sent) for sent in train_tweets]
    train_inputs = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_train]
    train_inputs = pad_sequences(train_inputs, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    # Create attention masks
    train_masks = []
    # Create a mask of 1s for each token followed by 0s for padding
    for seq in train_inputs:
      seq_mask = [float(i>0) for i in seq]
      train_masks.append(seq_mask)

    tokenized_test = [tokenizer.tokenize(sent) for sent in test_tweets]
    test_inputs = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_test]
    test_inputs = pad_sequences(test_inputs, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    # Create attention masks
    test_masks = []
    # Create a mask of 1s for each token followed by 0s for padding
    for seq in test_inputs:
      seq_mask = [float(i>0) for i in seq]
      test_masks.append(seq_mask)

    train_inputs = torch.tensor(train_inputs)
    test_inputs = torch.tensor(test_inputs)
    train_labels = torch.tensor(train_labels)
    test_labels = torch.tensor(test_labels)
    train_masks = torch.tensor(train_masks)
    test_masks = torch.tensor(test_masks)

    # Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
    # with an iterator the entire dataset does not need to be loaded into memory

    train_data = TensorDataset(train_inputs, train_masks, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    test_data = TensorDataset(test_inputs, test_masks, test_labels)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
    # Load XLNEtForSequenceClassification, the pretrained XLNet model with a single linear classification layer on top. 

    model = XLNetForMultiLabelSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=6)
    model.cuda()
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0}
    ]
    # This variable contains all of the hyperparemeter information that the training loop needs
    optimizer = AdamW(optimizer_grouped_parameters,
                        lr=lr)
    # Store the loss
    train_loss_set = []


    # trange is a tqdm wrapper around the normal python range
    for _ in trange(epochs, desc="Epoch"):
      
      
      # Training
      
      # Set our model to training mode (as opposed to evaluation mode)
      model.train()
      
      # Tracking variables
      tr_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0
      
      # Train the data for one epoch
      for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient

        optimizer.step()
        
        
        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

      print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # test
    # Put model in evaluation mode to evaluate loss on the test set
    model.eval()

    # Tracking variables 
    eval_loss=0
    nb_eval_steps, nb_eval_examples = 0, 0
    labels,predictions=[],[]
    # Evaluate data for one epoch
    for batch in test_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Telling the model not to compute or store gradients, saving memory and speeding up test
      with torch.no_grad():
        # Forward pass, calculate logit predictions
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = output[0]
      
      # Move logits and labels to CPU

      prob=torch.sigmoid(logits)
      # If probability greater than or equal to threshold T the tweet contains that emotion
      pred = (prob >= T).type(torch.FloatTensor)
      pred = pred.detach().cpu().numpy().astype(int).tolist()
      label_ids = b_labels.to('cpu').numpy().tolist()
      labels+=label_ids
      predictions+=pred
    labels=np.array(labels)
    predictions=np.array(predictions)
    recall_micro=recall_score(labels, predictions, average = "micro")
    recall_macro=recall_score(labels, predictions, average = "macro")
    precision_micro=precision_score(labels, predictions, average = "micro")
    precision_macro=precision_score(labels, predictions, average = "macro")
    f1_micro=f1_score(labels, predictions, average = "micro")
    f1_macro=f1_score(labels, predictions, average = "macro")
    model_metrics={("Precision","Micro"):precision_micro,("Precision","Macro"):precision_macro,
                  ("Recall","Micro"):recall_micro,("Recall","Macro"):recall_macro,
                  ("F1 score","Micro"):f1_micro,("F1 score","Macro"):f1_macro}
    confusion_matrix_ = multilabel_confusion_matrix(labels, predictions)
    print(model_metrics)
    return model_metrics, confusion_matrix_

In [0]:
# Grid Search for hyper-parameter tuning
from sklearn.model_selection import KFold
hps = [
(4, 32, 2e-5,0.5),
(2, 32, 2e-5,0.5),
(3, 32, 2e-5,0.5),
(3, 48, 2e-5,0.5),
(4, 32, 3e-5,0.3),
(4, 32, 3e-5,0.5),
(4, 32, 3e-5,0.8),
]
kfold_tweets = preprocess(train_tweets)
# 10-fold cross validation
cv = KFold(n_splits=10, random_state=42, shuffle=False)
val_scores = []
for epochs, batch_size, lr, T in hps:
  hp_scores = []
  for train_index, test_index in cv.split(kfold_tweets):
    tr_tweets = preprocess(train_tweets[train_index])
    va_tweets = preprocess(train_tweets[test_index])
    tr_labels = [train_labels[i] for i in train_index]
    va_labels = [train_labels[i] for i in test_index]
    hp_scores.append(xlnet_model(tr_tweets, va_tweets, tr_labels, va_labels, epochs, batch_size, lr, T)[0])
  hp_score_df = pd.DataFrame(hp_scores)
  hp_score = dict(hp_score_df.mean())
  val_scores.append((str(epochs)+", "+str(batch_size)+", "+str(lr)+", "+str(T), hp_score))

In [0]:
print(val_scores)

In [0]:
batch_size = 32
epochs = 4
lr = 3e-5
T = 0.5
# testing
clean_train_tweets, clean_test_tweets = (preprocess(train_tweets), preprocess(test_tweets))
test_scores = xlnet_model(clean_train_tweets, clean_test_tweets, train_labels, test_labels, epochs, batch_size, lr, T)

In [0]:
print(test_scores[1])